In [5]:
import numpy as np
import pandas as pd
import collections
from scipy import sparse

In [ ]:
!wget https://snap.stanford.edu/data/loc-gowalla_edges.txt.gz
!gzip -d loc-gowalla_edges.txt.gz
!wget https://snap.stanford.edu/data/loc-gowalla_totalCheckins.txt.gz
!gzip -d loc-gowalla_totalCheckins.txt.gz > /devnull

--2021-11-03 15:07:54--  https://snap.stanford.edu/data/loc-gowalla_edges.txt.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6351523 (6.1M) [application/x-gzip]
Saving to: ‘loc-gowalla_edges.txt.gz’

loc-gowalla_edges.t 100%[===================>]   6.06M  21.8MB/s    in 0.3s    

2021-11-03 15:07:54 (21.8 MB/s) - ‘loc-gowalla_edges.txt.gz’ saved [6351523/6351523]

--2021-11-03 15:07:55--  https://snap.stanford.edu/data/loc-gowalla_totalCheckins.txt.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 105470044 (101M) [application/x-gzip]
Saving to: ‘loc-gowalla_totalCheckins.txt.gz’

loc-gowalla_totalCh 100%[===================>] 100.58M  34.2MB/s    in 2.9s    

2021-11-03 15:07

In [2]:
graph = pd.read_csv('loc-gowalla_edges.txt', sep = '\t',  names=['source', 'dest'])
N = 500
source = graph['source'].values
source = source[source < N]
dest = graph['dest'].values
dest = dest[dest < N]
data = np.ones(source.shape[0], dtype=np.float64)
S = sparse.csr_matrix((data, (source, dest)), shape=(N, N))
S.setdiag(-1.0)

/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [3]:
%%capture --no-stdout

A = sparse.csr_matrix((np.zeros(source.shape[0], dtype=np.float32), (source, dest)), shape=(N, N))

R = sparse.csr_matrix((np.zeros(source.shape[0], dtype=np.float32), (source, dest)), shape=(N, N))

max_iterations = 29

for n in range(max_iterations):
    to_max = S + A
    tmp = to_max.copy()

    max_indices = np.asarray(np.argmax(to_max, axis=-1)).flatten()
    max_array = np.asarray(to_max[np.arange(N), max_indices]).flatten()

    tmp[np.arange(N), max_indices] = -np.inf

    prev_max_indices = np.asarray(np.argmax(tmp, -1)).flatten()
    prev_max_array = np.asarray(to_max[np.arange(N), prev_max_indices]).flatten()
    
    for idx, (i, j) in enumerate(zip(R.indptr, R.indptr[1:])):
        R.data[i:j] = S.data[i:j] - max_array[idx]
        if max_indices[idx] in R.indices[i:j]:
            R[idx, max_indices[idx]] = S[idx, max_indices[idx]] - prev_max_array[idx]

    A = R.copy()
    A.setdiag(0)
    A[A < 0] = 0
    sums = np.asarray(np.sum(A, axis=0)).flatten()
    sums_d = R.diagonal() + sums
    A.data = np.minimum(0, sums_d[A.indices] - A.data)
    A.setdiag(sums)

result = A + R
labels = [np.argmax(result[i]) for i in range(N)]

In [7]:
checkins = pd.read_csv('loc-gowalla_totalCheckins.txt', delimiter = '\t', names = ['user', 'time', 'latitude', 'longitude', 'location_id'])
users = checkins['user'].unique()
users = users[users < N]
np.random.shuffle(users)
test_users = users[:len(users) // 10]

clusters = defaultdict(list)
for user, i in enumerate(labels):
  if user not in test_users:
    clusters[i].append(user)

loc_counter = defaultdict(collections.Counter)
for i, vals in clusters.items():
    data = checkins[checkins['user'].isin(vals)]['location_id'].values
    loc_counter[i].update(data)

In [8]:
accuracies = []
for user in test_users:
    i = labels[user]
    accuracy = len(set([location_id for location_id, _ in loc_counter[i].most_common(10)])
                    & set(checkins[checkins['user'] == user]['location_id'].values)) / 10
    accuracies.append(accuracy)
print(f'accuracy: {np.mean(accuracies)}')

accuracy: 0.035555555555555556
